### Consigna del desafío 2

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.


### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto.  
Se utilizará el epub de Drácula para generar los embeddings, es decir, que los vectores tendrán la forma en función de las líneas del libro.

In [1]:
import pandas as pd
import os
import requests 
from gensim.models import Word2Vec

### Datos
Utilizo como dataset el epub de Drácula

In [2]:
# Funcion auxiliar para descargar el dataset
def download_dataset(dataset_url: str, target_filename: str, force: bool = False):
    if os.path.exists(target_filename) and not force:
        print("Dataset folder already exists, nothing downloaded.")
        return

    os.makedirs(os.path.dirname(target_filename), exist_ok=True)
    try:
        with requests.get(dataset_url, stream=True) as response:
            response.raise_for_status()  # Raise an exception for bad status codes

            with open(target_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"File '{target_filename}' downloaded successfully.")
    except requests.exceptions.RequestException as e:
        raise(Exception(f"Error downloading file: {e}"))

In [21]:
# Descargar el dataset
dataset = "datasets/dracula.txt"
download_dataset("https://www.gutenberg.org/cache/epub/345/pg345.txt", dataset)

File 'datasets/dracula.txt' downloaded successfully.


In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv(dataset, sep='/n', header=None, engine="python")
df.head()

,0
0,The Project Gutenberg eBook of Dracula
1,This ebook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 13373


### 1 - Preprocesamiento

In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")
sentence_tokens = []
for _, row in df[:None].iterrows():
    # Agrego como preprocesamiento que convierta todas las palabras a minúsculas
    doc = nlp(row[0].lower())
    sentence_tokens.append([token.text for token in doc])

In [7]:
# Demos un vistazo
print(sentence_tokens[:5])
print(f"Sentence tokens totales: {len(sentence_tokens)}")

[['the', 'project', 'gutenberg', 'ebook', 'of', 'dracula'], ['this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and'], ['most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions'], ['whatsoever', '.', 'you', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're', '-', 'use', 'it', 'under', 'the', 'terms'], ['of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online']]
Sentence tokens totales: 13373


### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=100,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1,           # modelo 0:CBOW  1:skipgram
                     seed=42)

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 13373


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2523


### 3 - Entrenar embeddings

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1358544.0
Loss after epoch 1: 1004671.25
Loss after epoch 2: 905189.75
Loss after epoch 3: 903380.75
Loss after epoch 4: 838546.75
Loss after epoch 5: 831034.0
Loss after epoch 6: 825068.0
Loss after epoch 7: 821244.5
Loss after epoch 8: 814963.0
Loss after epoch 9: 779518.0
Loss after epoch 10: 769732.0
Loss after epoch 11: 766161.0
Loss after epoch 12: 759601.0
Loss after epoch 13: 759558.0
Loss after epoch 14: 753447.0
Loss after epoch 15: 750565.0
Loss after epoch 16: 748332.0
Loss after epoch 17: 742732.0
Loss after epoch 18: 740033.0
Loss after epoch 19: 740677.0
Loss after epoch 20: 719230.0
Loss after epoch 21: 706706.0
Loss after epoch 22: 705298.0
Loss after epoch 23: 701392.0
Loss after epoch 24: 702912.0
Loss after epoch 25: 702628.0
Loss after epoch 26: 698680.0
Loss after epoch 27: 696498.0
Loss after epoch 28: 695136.0
Loss after epoch 29: 693076.0
Loss after epoch 30: 690884.0
Loss after epoch 31: 686020.0
Loss after epoch 32: 687714.0
Loss after epo

(11910197, 19591000)

### 4 - Ensayo

In [14]:
def comparador(words: list[str], positive: bool = True):
    comparaciones = []
    for word in words:
        arg = {"positive": [word]} if positive else {"negative": [word]}
        comp = w2v_model.wv.most_similar(topn=4, **arg)
        comparaciones.append({"Palabra": word, 
                            "Similitud 1": f"{comp[0][0]} ({comp[0][1]:.3f})", "Similitud 2": f"{comp[1][0]} ({comp[1][1]:.3f})", 
                            "Similitud 3": f"{comp[2][0]} ({comp[2][1]:.3f})", "Similitud 4": f"{comp[3][0]} ({comp[3][1]:.3f})"})
    df_comp = pd.DataFrame(comparaciones)
    print(f"Similitudes: {'positivas' if positive else 'negativas'}")
    display(df_comp)

#### Palabras relacionadas positivamente

In [15]:
words = ["dracula", "project", "teeth", "transfusion"]
comparador(words, positive=True)

Similitudes: positivas


,Palabra,Similitud 1,Similitud 2,Similitud 3,Similitud 4
0,dracula,criminal (0.493),madness (0.481),silent (0.473),candy (0.431)
1,project,gutenberg (0.739),mission (0.653),volunteers (0.607),distributing (0.591)
2,teeth,gums (0.540),wax (0.512),sharper (0.480),size (0.462)
3,transfusion,plenty (0.543),blood (0.462),operation (0.460),bloom (0.454)


#### Conclusiones:
La palabra dracula tiene una buena similitud con criminal, madness, silent y candy.  
Puedo suponer hay similitud semántica y que es usada en contextos similares.  
Criminal, madness y silent pueden tener sentido que se mencionen en contextos similares.  
Candy no me parece tan lógico, pero buscando info, se trata de "Mitchell, Sons & Candy", una empresa de bienes raíces ficticia en el que se investigan las propiedades del conde dracula.  

La palabra project, tiene muy buena similitud con gutenberg, mission, volunteers y distributing.  
Si bien no pertenece a la novela, el epub tiene una referencia al final al proyecto gutenberg, con terminos de licencia y uso del epub.  
Es por eso que todas estas palabras pueden estar siendo usadas en contextos similares.  

La palabra teeth, tiene buena similitud con gums, wax, sharper y size.  
Puedo suponer que al hablar de dientes, se describan algunas caracteristicas, y se hable en referencia a estas en contextos similares.  

La palabra transfusion, tiene similitud con plenty, blood, operation y bloom.  
Con blood y operation tiene bastante sentido que aparezcan en contextos similares.  

#### Palabras relacionadas negativamente

In [16]:
words = ["love", "dracula", "night", "brain"]
comparador(words, positive=False)

Similitudes: negativas


,Palabra,Similitud 1,Similitud 2,Similitud 3,Similitud 4
0,love,herself (0.144),conscious (0.089),entering (0.069),rested (0.066)
1,dracula,slept (0.123),hours (0.099),cried (0.090),makes (0.087)
2,night,voluptuous (0.090),earnest (0.079),trembled (0.067),draw (0.060)
3,brain,held (0.140),met (0.124),leaned (0.103),waiting (0.103)


#### Conclusiones
Analizar palabras relacionadas negativamente en este caso es un poco mas complejo.  
Puedo pensar que love y herself aparecen como contrapuestas, que nunca se habla de uno mismo en contextos donde se habla de amor.  
O que cuando se habla de amor, no se habla de conciencia o consciente.  
Parece que amor y descansado estarían contrapuestos.  

Para dracula, aparece como contrapuesto o negativo durmió, puedo pensar que dracula no aparece dormido.  
O que en los contextos donde se habla de drácula, no se mencionan horas, o llantos.  

En el caso de night, no logro asociar negativamente los términos.  
Noche y serio, noche y tembló, noche y dibujo aparecerían en contextos contrapuestos.

Por último, para la palabra cerebro, aparecen como contextos negativos retenido, reunido, inclinado, y esperando.  
No parecen tener similitud positiva como para pensar que estan mal, pero tampoco encuentro mucha similitud negativa como para afirmar que estan bien.

Busqué con distintos términos, no fue facil encontrar casos muy representativos

#### Test de analogías

In [17]:
# Test de analogías

# Ajo - proteje + sangre = riesgo
result = w2v_model.wv.most_similar(positive=['garlic', 'blood'], negative=['protect'], topn=1)
print(result)


[('risk', 0.4035259783267975)]


#### Conclusion
Para el test de analogía, el ejemplo planteado podría tener cierta lógica.  
Si el ajo proteje, la sangre implica riesgo.  
El ajo es a protección como la sangre es a riesgo.  
Desde el punto de vista de que el ajo proteje de drácula, la presencia de sangre sería el riesgo de haber sido mordido por drácula.

### 5 - Visualizar agrupación de vectores

In [18]:
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=42)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=100
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()
# fig.show(renderer="colab") # esto para plotly en colab

Palabras como van y helsing se observan en (-6, -1) y estan superpuestas. Eso significa una fuerte relación positiva entre ellas.  
Lucy o night aparecen bastante separadas del resto. Esto significaría una relación negativa con el resto de los términos.

Esta representación es parcial y seguramente esta perdiendo muchos detalles, ya que solo se muestran 100 palabras de todo el corpus.

In [20]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
# fig.show(renderer="colab") # esto para plotly en colab
fig.show()

En la gráfica 3D, vemos como las palabras it, to y all estan muy separadas del resto de términos, indicando una relación muy negativa respecto al resto de términos.


### Conclusiones generales

En el trabajo se observaron similitudes semánticas, positivas y negativas.  
En general al realizar el análisis intuitivo, costó pensar estas relaciones, pero puedo suponer que en parte se debe al texto utilizado, y que este texto tiene oraciones cortadas en distintas líneas.  
Estoy tomando cada línea como un documento, por lo que limita la interpretación.  
Si se considerara cada oración completa, por ejemplo, separada por stop words, posiblemente el contexto se interpretaría mejor.  
Probé cambiando la ventana de Word2Vec a valores como 3, 4 y 5, pero no obtuve mejores resultados.  
Para algunos ejemplos parecia mejor el contexto, pero para otros no.
